In [9]:
import logging
import torch
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import PyPDF2

# pip install transformers sentence-transformers PyPDF2 torch

def setup_logging(log_path="C:\\Users\\Naman\\Desktop\\jupyter notebook\\support_bot_log.txt"):
    """Setup logging to log both to file and console."""
    logging.basicConfig(
        filename=log_path,
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        filemode="w",
        force=True
    )
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    console_handler.setFormatter(formatter)
    logging.getLogger().addHandler(console_handler)


class SupportBotAgent:
    def __init__(self, document_path):
        """Initialize the support bot with a document."""
        setup_logging()
        self.qa_model = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        self.document_text = self.load_document(document_path)
        self.sections = self.split_document_into_sections(self.document_text)
        
        if not self.sections:
            logging.error("No valid sections found in the document.")
            raise ValueError("Document appears to be empty or unreadable.")
        
        self.section_embeddings = self.embedder.encode(self.sections, convert_to_tensor=True)
        logging.info(f"Loaded document: {document_path} with {len(self.sections)} sections.")

    def load_document(self, path):
        """Load text from a given file (supports .txt and .pdf)."""
        try:
            if path.endswith('.pdf'):
                return self.extract_text_from_pdf(path)
            with open(path, 'r', encoding='utf-8') as file:
                return file.read()
        except FileNotFoundError:
            logging.error(f"File not found: {path}")
            raise

    def extract_text_from_pdf(self, pdf_path):
        """Extract text from a PDF document."""
        text = ""
        try:
            with open(pdf_path, "rb") as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
            return text.strip() or "No text extracted from PDF."
        except Exception as e:
            logging.error(f"Error reading PDF: {e}")
            return "No text extracted from PDF."

    def split_document_into_sections(self, text):
        """Splits document text into sections for efficient search."""
        return [section.strip() for section in text.split('\n\n') if section.strip()]

    def find_relevant_section(self, query, threshold=0.4):
        """Find the most relevant section of the document based on query similarity."""
        query_embedding = self.embedder.encode(query, convert_to_tensor=True)
        similarities = util.cos_sim(query_embedding, self.section_embeddings)[0]
        best_idx = similarities.argmax().item()
        best_score = similarities[best_idx].item()

        if best_score < threshold:
            logging.info(f"No highly relevant section found for query: {query} (score: {best_score:.2f})")
            return None

        logging.info(f"Relevant section found for query: {query} (score: {best_score:.2f})")
        return self.sections[best_idx]

    def answer_query(self, query):
        """Generate an answer based on the most relevant section."""
        context = self.find_relevant_section(query)
        if not context:
            return "I'm sorry, I couldn't find relevant information in the document. Please contact support."

        result = self.qa_model(question=query, context=context)
        confidence_score = result.get("score", 0)

        if confidence_score < 0.3:
            logging.info(f"Low confidence ({confidence_score:.2f}) for query: {query}")
            return "I'm not sure about that. Please try rephrasing or contact support."

        return result["answer"]

    def get_feedback(self, response):
        """Prompt the user for feedback on the bot's response."""
        feedback = input(f"Was this response helpful? (yes/no/more details): ").strip().lower()
        return "not helpful" if feedback in ["no", "not helpful"] else "too vague" if feedback in ["more details", "too vague"] else "good"

    def adjust_response(self, query, response, feedback):
        """Adjust response based on user feedback."""
        if feedback == "too vague":
            context = self.find_relevant_section(query)
            return f"{response} (Additional Info: {context[:100]}...)" if context else response

        if feedback == "not helpful":
            alternate_context = self.find_relevant_section(query, threshold=0.3)
            if alternate_context:
                return self.qa_model(question=query, context=alternate_context)["answer"]
            return response + " I'm sorry, I couldn't find more details."
        
        return response

    def run(self, queries):
        """Run the bot, processing each query and refining responses based on feedback."""
        for query in queries:
            logging.info(f"Processing query: {query}")
            response = self.answer_query(query)
            print(f"\nInitial Response to '{query}': {response}")

            for _ in range(2):  # Allow up to two rounds of refinement
                feedback = self.get_feedback(response)
                if feedback == "good":
                    break
                response = self.adjust_response(query, response, feedback)
                print(f"Updated Response to '{query}': {response}")


if __name__ == "__main__":
    bot = SupportBotAgent("faq.txt")  # Load document
    sample_queries = [
        "How do I reset my password?",
        "What’s the refund policy?",
        "How do I contact support?",
        "How do I fly to the moon?"  # Out-of-scope query
    ]
    bot.run(sample_queries)


2025-03-10 20:22:21,639 - INFO - Use pytorch device_name: cpu
2025-03-10 20:22:21,639 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-03-10 20:22:28,097 - INFO - Loaded document: faq.txt with 3 sections.
2025-03-10 20:22:28,113 - INFO - Processing query: How do I reset my password?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-03-10 20:22:28,204 - INFO - Relevant section found for query: How do I reset my password? (score: 0.80)


...


Initial Response to 'How do I contact support?': Email us at support@example.com
Was this response helpful? (yes/no/more details): yes


2025-03-10 20:23:15,374 - INFO - Processing query: How do I fly to the moon?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-03-10 20:23:15,488 - INFO - No highly relevant section found for query: How do I fly to the moon? (score: 0.02)



Initial Response to 'How do I fly to the moon?': I'm sorry, I couldn't find relevant information in the document. Please contact support.
Was this response helpful? (yes/no/more details): yes
